In [1]:
import requests
import json
import numpy as np
import pandas as pd
# AlphaVantage Demo IBM API
url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=IBM&apikey=demo'
response = requests.get(url)
# Load text to clean json object
json_load = json.loads(response.text)
# 'Time Series (Daily)' consists the time stamps for the demo
df = pd.DataFrame(json_load['Time Series (Daily)']).T
# Rename column names & change datetime to pandas datetime
df.reset_index(inplace=True)
df.columns = ['datetime', 'open', 'high', 'low', 'close', 'volume']
df['datetime'] = pd.to_datetime(df['datetime'])
display(df)

,datetime,open,high,low,close,volume
0,2022-07-01,141.0000,141.6700,139.2600,141.1200,4012106
1,2022-06-30,139.5800,142.4600,139.2800,141.1900,4878020
2,2022-06-29,142.7400,143.5213,139.5000,140.7100,4161491
3,2022-06-28,142.9200,144.1550,141.3200,141.8600,4065202
4,2022-06-27,142.2600,143.8300,141.9500,142.8000,3935968
...,...,...,...,...,...,...
95,2022-02-14,132.5900,132.6500,129.0700,130.1500,5345289
96,2022-02-11,133.9000,134.7099,132.3800,132.6900,4176155
97,2022-02-10,135.4700,136.5600,133.1700,133.5200,5978640
98,2022-02-09,137.8400,138.3500,136.8300,137.7900,5393478


In [2]:
WINDOW = 30
df['sma'] = df['close'].rolling(WINDOW).mean()
df['std'] = df['close'].rolling(WINDOW).std(ddof = 0)
display(df)

,datetime,open,high,low,close,volume,sma,std
0,2022-07-01,141.0000,141.6700,139.2600,141.1200,4012106,NaN,NaN
1,2022-06-30,139.5800,142.4600,139.2800,141.1900,4878020,NaN,NaN
2,2022-06-29,142.7400,143.5213,139.5000,140.7100,4161491,NaN,NaN
3,2022-06-28,142.9200,144.1550,141.3200,141.8600,4065202,NaN,NaN
4,2022-06-27,142.2600,143.8300,141.9500,142.8000,3935968,NaN,NaN
...,...,...,...,...,...,...,...,...
95,2022-02-14,132.5900,132.6500,129.0700,130.1500,5345289,126.271667,2.796708
96,2022-02-11,133.9000,134.7099,132.3800,132.6900,4176155,126.312333,2.879641
97,2022-02-10,135.4700,136.5600,133.1700,133.5200,5978640,126.384667,3.028670
98,2022-02-09,137.8400,138.3500,136.8300,137.7900,5393478,126.669333,3.626871


In [3]:
import plotly.graph_objects as go
from plotly.offline import iplot
from plotly.subplots import make_subplots

# Create subplots with 2 rows; top for candlestick price, and bottom for bar volume
fig = make_subplots(rows = 2, cols = 1, shared_xaxes = True, subplot_titles = ('IBM', 'Volume'), vertical_spacing = 0.1, row_width = [0.2, 0.7])

# ----------------
# Candlestick Plot
fig.add_trace(go.Candlestick(x = df['datetime'],
                             open = df['open'],
                             high = df['high'],
                             low = df['low'],
                             close = df['close'], showlegend=False,
                             name = 'candlestick'),
              row = 1, col = 1)

# Moving Average
fig.add_trace(go.Scatter(x = df['datetime'],
                         y = df['sma'],
                         line_color = 'black',
                         name = 'sma'),
              row = 1, col = 1)

# Upper Bound
fig.add_trace(go.Scatter(x = df['datetime'],
                         y = df['sma'] + (df['std'] * 2),
                         line_color = 'gray',
                         line = {'dash': 'dash'},
                         name = 'upper band',
                         opacity = 0.5),
              row = 1, col = 1)

# Lower Bound fill in between with parameter 'fill': 'tonexty'
fig.add_trace(go.Scatter(x = df['datetime'],
                         y = df['sma'] - (df['std'] * 2),
                         line_color = 'gray',
                         line = {'dash': 'dash'},
                         fill = 'tonexty',
                         name = 'lower band',
                         opacity = 0.5),
              row = 1, col = 1)


# ----------------
# Volume Plot
fig.add_trace(go.Bar(x = df['datetime'], y = df['volume'], showlegend=False), 
              row = 2, col = 1)

# Remove range slider; (short time frame)
fig.update(layout_xaxis_rangeslider_visible=False)

# Stock data has gaps in dates, specifically in weekends and holidays
# create a list of dates that are NOT included from start to end
date_gaps = [date for date in pd.date_range(start = '2020-12-21', end = '2021-05-14') if date not in df['datetime'].values]

# Update Xaxes 
fig.update_xaxes(rangebreaks = [dict(values = date_gaps)])

fig.show();